---
preview: LangChain support for HANA KG engine

In [1]:
# !pip install ../whl/langchain_hana-0.1.0-py3-none-any.whl

In [3]:
import langchain
import langchain_community
import langchain_hana
print('langchain version:', langchain.__version__)
print('langchain_community version:', langchain_community.__version__)
print('langchain_hana version:', langchain_hana.__version__)

langchain version: 0.3.19
langchain_community version: 0.3.18
langchain_hana version: 0.1.0


In [4]:
from langchain_hana import HanaRdfGraph
from langchain_hana import HanaSparqlQAChain

In [5]:
# connect to HANA Cloud
from hdbcli import dbapi
connection = dbapi.connect(key='KGE2')

---

In [12]:
# let's create a LangChain Graph Store by referring to a named graph and its ontology
ontology_uri = "wiki_movies_ontology"
graph_uri = "wiki_movies"

hkg = HanaRdfGraph(
    connection=connection, 
    ontology_uri=ontology_uri, 
    graph_uri=graph_uri
    )

# print(hkg.get_schema)


In [7]:
# sure we can run SPARQL queries
sparql_query = '''
	PREFIX demo: <http://kg.demo.sap.com/>
	PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
	SELECT ?movie ?actor ?actorLabel
	FROM <wiki_movies>
	WHERE {
		?movie a demo:Film ;
	  		demo:title "The Matrix" .
		?actor demo:acted_in ?movie ;
	  		rdfs:label ?actorLabel .
	} LIMIT 10
    '''

# hkg.query(query=sparql_query,content_type='application/sparql-results+json')

hkg.convert_csv_response_to_dataframe(hkg.query(query=sparql_query))

,movie,actor,actorLabel
0,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q21870719,Rowan Witt
1,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q193048,Laurence Fishburne
2,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q258604,David Aston
3,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q4882899,Belinda McClory
4,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q3324448,Anthony Ray Parker
5,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q42204,Hugo Weaving
6,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q862673,Bill Young
7,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q296883,Joe Pantoliano
8,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q43416,Keanu Reeves
9,http://www.wikidata.org/entity/Q83495,http://www.wikidata.org/entity/Q455234,Gloria Foster


---
Create a LangChain SPARQL QA Chain<br>
The SPARQL QA Chain translates a natural language query into a SPARQL query which fits the KGs model

In [13]:
# we need an LLM to generate the SAPRQL queries
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
llm = ChatOpenAI(
    proxy_model_name="gpt-4o", temperature=0
)

In [14]:
# let's create a SPARQL QA Chain
hkg_qa_chain = HanaSparqlQAChain.from_llm(
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    graph=hkg
    )

In [10]:
query = "Which directors played most often a role in their film? Which roles did they play and what are the film titles?"
# query = "which are the two latest film in the genre 'action film' and who directed them?"

output = hkg_qa_chain.invoke(query)



> Entering new HanaSparqlQAChain chain...
Generated SPARQL:
```
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?directorName ?role ?filmTitle
WHERE {
    ?director a <http://kg.demo.sap.com/Director> .
    ?director rdfs:label ?directorName .
    ?director <http://kg.demo.sap.com/directed> ?film .
    ?film <http://kg.demo.sap.com/title> ?filmTitle .
    ?film <http://kg.demo.sap.com/castMemberStatement> ?castStatement .
    ?castStatement <http://kg.demo.sap.com/castMember> ?director .
    ?castStatement <http://kg.demo.sap.com/characterRole> ?role .
}
```
Final SPARQL (with FROM clause injected):

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?directorName ?role ?filmTitle

FROM <wiki_movies>
WHERE {
    ?director a <http://kg.demo.sap.com/Director> .
    ?director rdfs:label ?directorName .
    ?director <http://kg.demo.sap.com/directed> ?film .
    ?film <http://kg.demo.sap.com/title> ?filmTitle .
    ?film <http://kg.demo.sap.com/castMemberStatement> ?

In [11]:
print('User input: ' + output['query'])
print('LLM response: ' + output['result'])

User input: Which directors played most often a role in their film? Which roles did they play and what are the film titles?
LLM response: The directors who played roles most often in their films, according to the provided information, are:

1. **Kevin Smith**: 
   - Role: Silent Bob
     - Films: "Jay and Silent Bob Strike Back," "Dogma," "Chasing Amy," "Mallrats"

2. **Clint Eastwood**:
   - Roles: 
     - Walt Kowalski in "Gran Torino"
     - Gunnery Sergeant Tom Highway in "Heartbreak Ridge"
     - Terry McCaleb in "Blood Work"
     - Chief 'Red' Garnett in "A Perfect World"
     - William "Bill" Munny in "Unforgiven"
     - Harry Callahan in "Sudden Impact"
     - Mitchell Gant in "Firefox"

3. **Mel Brooks**:
   - Roles:
     - Friar Tuck and Rabbi Tuckman in "Robin Hood: Men in Tights"
     - Skroob and Yogurt in "Spaceballs"

4. **Taika Waititi**:
   - Roles:
     - Korg and Korg / Old Kronan God in "Thor: Love and Thunder" and "Thor : Amour et tonnerre"
     - Adolf Hitler in "

---

In [26]:
# sure we can run SPARQL queries
sparql_query = '''
PREFIX kg: <http://kg.demo.sap.com/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT *

FROM <wiki_movies>
WHERE {
    ?director a kg:Director ;
              kg:directed ?film ;
              rdfs:label ?directorName .
    
    ?film kg:publicationDate ?pubDate ;
          kg:title ?filmTitle ;
          kg:castMemberStatement ?castStatement .
    ?castStatement kg:castMember ?director ;
                   kg:characterRole ?role .
    FILTER (YEAR(?pubDate) = 2022)
}'''
hkg.convert_csv_response_to_dataframe(hkg.query(query=sparql_query))


,director,film,directorName,pubDate,filmTitle,castStatement,role
0,http://www.wikidata.org/entity/Q5236475,http://www.wikidata.org/entity/Q99900595,David Leitch,2022-08-04T00:00:00Z,Bullet Train,http://www.wikidata.org/entity/statement/Q9990...,Jeff Zufelt
1,http://www.wikidata.org/entity/Q5236475,http://www.wikidata.org/entity/Q99900595,David Leitch,2022-08-03T00:00:00Z,Bullet Train,http://www.wikidata.org/entity/statement/Q9990...,Jeff Zufelt
2,http://www.wikidata.org/entity/Q5236475,http://www.wikidata.org/entity/Q99900595,David Leitch,2022-07-18T00:00:00Z,Bullet Train,http://www.wikidata.org/entity/statement/Q9990...,Jeff Zufelt
3,http://www.wikidata.org/entity/Q2388576,http://www.wikidata.org/entity/Q65768604,Taika Waititi,2022-07-08T00:00:00Z,Thor: Love and Thunder,http://www.wikidata.org/entity/statement/Q6576...,Korg
4,http://www.wikidata.org/entity/Q2388576,http://www.wikidata.org/entity/Q65768604,Taika Waititi,2022-07-08T00:00:00Z,Thor : Amour et tonnerre,http://www.wikidata.org/entity/statement/Q6576...,Korg
5,http://www.wikidata.org/entity/Q2388576,http://www.wikidata.org/entity/Q65768604,Taika Waititi,2022-07-08T00:00:00Z,Thor: Love and Thunder,http://www.wikidata.org/entity/statement/Q6576...,Korg / Old Kronan God
6,http://www.wikidata.org/entity/Q2388576,http://www.wikidata.org/entity/Q65768604,Taika Waititi,2022-07-08T00:00:00Z,Thor : Amour et tonnerre,http://www.wikidata.org/entity/statement/Q6576...,Korg / Old Kronan God
7,http://www.wikidata.org/entity/Q200355,http://www.wikidata.org/entity/Q100801718,Olivia Wilde,2022-09-23T00:00:00Z,Don't Worry Darling,http://www.wikidata.org/entity/statement/Q1008...,Mary
8,http://www.wikidata.org/entity/Q200355,http://www.wikidata.org/entity/Q100801718,Olivia Wilde,2022-09-22T00:00:00Z,Don't Worry Darling,http://www.wikidata.org/entity/statement/Q1008...,Mary
9,http://www.wikidata.org/entity/Q55294,http://www.wikidata.org/entity/Q56816969,Kenneth Branagh,2022-02-10T00:00:00Z,Death on the Nile,http://www.wikidata.org/entity/statement/Q5681...,Hercule Poirot
